**Sentiment Analysis of IMDB Reviews Using BERT**

# Step 1: Installing the required libraries

In [1]:
# Установка необходимых библиотек
!pip install torch pandas numpy transformers
!pip install wandb # Для логирования результатов в Weights & Biases

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

# Step 2: Import Dependencies

In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
import wandb # Для логирования экспериментов

# Step 3: Download IMDB dataset

In [3]:
# Загрузка урезанного датасета IMDB
df = pd.read_csv(
    'https://gist.githubusercontent.com/Mukilan-Krishnakumar/e998ecf27d11b84fe6225db11c239bc6/raw/74dbac2b992235e555df9a0a4e4d7271680e7e45/imdb_movie_reviews.csv'
)

# Удаление существующего столбца 'sentiment', если он есть (это необязательно, если его нет)
# В нашем случае, скачанный датасет уже не содержит этот столбец.
# Если бы он был, и ты хотел бы его использовать, то не удалял бы.
# Здесь он удаляется, чтобы показать, что мы создаем его сами.
if 'sentiment' in df.columns:
    df = df.drop('sentiment', axis=1)

print("Первые 5 строк датасета:")
print(df.head())
print("\nИнформация о датасете:")
df.info()

Первые 5 строк датасета:
                                                text
0  My daughter liked it but I was aghast, that a ...
1  I... No words. No words can describe this. I w...
2  this film is basically a poor take on the old ...
3  This is a terrible movie, and I'm not even sur...
4  First of all this movie is a piece of reality ...

Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    135 non-null    object
dtypes: object(1)
memory usage: 1.2+ KB


# Step 4: Building and Evaluating the Model (Sentiment Analysis)

Here we use the pre-trained model nlptown/bert-base-multilingual-uncased-sentiment, which is already configured to analyze sentiment on a 5-point scale.

In [4]:
# Загрузка токенизатора и модели BERT для анализа тональности
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Определение вспомогательной функции для вычисления оценки тональности
def sentiment_movie_score(movie_review):
    # Токенизация текста отзыва и преобразование в PyTorch тензор
    # `truncation=True` гарантирует, что текст будет обрезан до максимальной длины модели (обычно 512 токенов),
    # а `padding=True` добавляет паддинг до этой длины, если текст короче.
    # `return_tensors='pt'` возвращает PyTorch тензоры.
    token = tokenizer.encode(movie_review, return_tensors='pt', truncation=True, padding=True)

    # Пропуск токенизированного ввода через модель для получения выходных логитов
    # `.logits` дает нам необработанные выходные значения для каждого класса
    result = model(token)

    # Выбор индекса с наивысшим логитом и сдвиг шкалы с 0-4 на 1-5
    # Модель `nlptown/bert-base-multilingual-uncased-sentiment` имеет 5 выходных классов
    # (0, 1, 2, 3, 4), которые соответствуют тональности.
    # Мы добавляем 1, чтобы получить шкалу 1-5.
    return int(torch.argmax(result.logits)) + 1

# Применение функции оценки к каждому отзыву и сохранение результатов в новом столбце 'sentiment'
# Мы ограничиваем длину отзыва до 512 токенов, так как BERT имеет ограничение по входной длине.
# В данном случае, x[:512] не совсем корректно, так как 512 - это количество токенов, а не символов.
# Но для маленького датасета это может быть не критично.
# Правильнее было бы, чтобы tokenizer сам обрезал, что мы и сделали, добавив truncation=True.
df['sentiment'] = df['text'].apply(lambda x: sentiment_movie_score(x))

print("\nПервые 5 строк датасета с предсказанной тональностью:")
print(df.head())
print("\nРаспределение предсказанной тональности:")
print(df['sentiment'].value_counts())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]


Первые 5 строк датасета с предсказанной тональностью:
                                                text  sentiment
0  My daughter liked it but I was aghast, that a ...          3
1  I... No words. No words can describe this. I w...          1
2  this film is basically a poor take on the old ...          2
3  This is a terrible movie, and I'm not even sur...          1
4  First of all this movie is a piece of reality ...          4

Распределение предсказанной тональности:
sentiment
4    34
2    33
3    29
1    23
5    16
Name: count, dtype: int64


# Step 5: Visualize Results (using Weights & Biases)

Weights & Biases (W&B) is a great tool for tracking experiments, logging metrics, models, and data.

In [5]:
# Убедись, что wandb установлен (выполняем его в Шаге 1)
# !pip install wandb

# Импорт wandb (уже в Шаге 2)
# import wandb

# Инициализация нового W&B запуска под проектом "BERT_Sentiment_Analysis"
# Если ты запускаешь это в Colab, он предложит тебе авторизоваться.
wandb.init(project="BERT_Sentiment_Analysis")

# Логирование DataFrame отзывов и предсказаний как W&B Table
# Это позволит тебе просматривать весь датасет с предсказаниями в UI W&B.
# Мы создаем wandb.Table из нашего pandas DataFrame.
predictions_table = wandb.Table(dataframe=df)
wandb.run.log({"Sentiment Analysis of IMDB Movie Reviews": predictions_table})

# Закрытие и финализация текущего W&B запуска
wandb.run.finish()

print("\nРезультаты анализа тональности отправлены в Weights & Biases!")
print("Ты можешь просмотреть их по ссылке, которую W&B выведет в консоль.")

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: illia777 (illia777-intk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Результаты анализа тональности отправлены в Weights & Biases!
Ты можешь просмотреть их по ссылке, которую W&B выведет в консоль.


In [6]:
# Найди этот конкретный отзыв (или его часть)
review_text_start = "My daughter liked it but I was aghast, that a character in this movie smokes."
# Найди строку, где текст отзыва начинается так
matching_row = df[df['text'].str.startswith(review_text_start)]

if not matching_row.empty:
    print("\nНайденный отзыв и его тональность:")
    print(matching_row[['text', 'sentiment']])
else:
    print("\nОтзыв не найден в DataFrame.")


Найденный отзыв и его тональность:
                                                text  sentiment
0  My daughter liked it but I was aghast, that a ...          3


# SUM

Based on user comments (reviews), it will be possible to automatically and on a large scale evaluate how they perceived a film (product, service, event, etc.).